In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras as keras
from focal_loss import SparseCategoricalFocalLoss

import onnx

2021-11-25 06:59:52.672693: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
model_path = "../model/antispoofing.h5"
model = tf.keras.models.load_model(model_path, custom_objects={"lr": lambda x: x, "SparseCategoricalFocalLoss": SparseCategoricalFocalLoss}, compile=False)

2021-11-25 06:59:53.520088: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-25 06:59:53.576034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1079] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 06:59:53.576449: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1742] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2021-11-25 06:59:53.576485: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-25 06:59:53.579335: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-11-25 06:59:53.580304: I tensorflow/stream_executor/plat

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 18)      504       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 18)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 18)      72        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 18)      2934      
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 18)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 18)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 18)        7

In [4]:
def add_permute_and_save(non_permutted, path = "../model/permuted_antispoofing.h5"):
    permuted_model = keras.Sequential([
        layers.Permute((2, 3, 1), input_shape=non_permutted.input_shape[1:][::-1]),
        model,
    ]);
    if path is not None:
        permuted_model.save(path)
    return permuted_model

In [5]:
permuted_model = add_permute_and_save(model, path = None)

In [6]:
# saving permuted model in SavedModel format
permuted_model.save("../model/permuted_antispoofing")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-11-25 06:59:55.364463: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../model/permuted_antispoofing/assets


In [7]:
!python -m tf2onnx.convert --saved-model ../model/antispoofing --output ../model/antispoofing.onnx

2021-11-25 06:59:57.526223: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-11-25 06:59:58.480192: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-25 06:59:58.499426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1079] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 06:59:58.499704: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1742] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.79GiB d

In [8]:
# setting batch dimension of input and output to -1 in the model, cuz it seems to give unk__48 when opening the model in netron

onnx_model = onnx.load("../model/permuted_antispoofing.onnx")

In [9]:
def update_dim(model):
    inputs = model.graph.input
    outputs = model.graph.output

    for inp in inputs:
        inp.type.tensor_type.shape.dim[0].dim_value = -1
    
    for output in outputs:
        output.type.tensor_type.shape.dim[0].dim_value = -1

In [10]:
update_dim(onnx_model)
onnx.save(onnx_model, "../model/permuted_antispoofing.onnx")